# This code is written by Hemanth Chebiyam.
## Email: hc3746@rit.edu

In [51]:
import os
import sys
import random
import operator
import numpy as np

In [52]:
class Environment:
    def __init__(self, Ny=8, Nx=8):
        # State space
        self.Ny = Ny
        self.Nx = Nx
        self.state_dim = (Ny, Nx)
        
        # Action space
        self.action_dim = (4,)
        self.action_dict = {"UP": 0, "RIGHT": 1, "DOWN": 2, "LEFT": 3}
        self.action_coords = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        
        # Rewards
        self.R = self._build_rewards()

    def reset(self):
        # Reset agent state
        self.state = (0, 0)
        return self.state

    def step(self, action):
        # Evolve agent state
        state_next = (self.state[0] + self.action_coords[action][0],
                      self.state[1] + self.action_coords[action][1])
        # Collect reward
        reward = self.R[self.state + (action,)]
        # Terminate at goal state (bottom-right grid corner)
        done = (state_next[0] == self.Ny - 1) and (state_next[1] == self.Nx - 1)
        # Update state
        self.state = state_next
        return state_next, reward, done

    def allowed_actions(self):
        # List of actions allowed
        actions_allowed = []
        # Based on agent grid location
        y, x = self.state[0], self.state[1]
        if y > 0: # top-boundary
            actions_allowed.append(self.action_dict["UP"])
        if y < self.Ny - 1: # bottom-boundary
            actions_allowed.append(self.action_dict["DOWN"])
        if x > 0: # left-boundary
            actions_allowed.append(self.action_dict["LEFT"])
        if x < self.Nx - 1: # right-boundary
            actions_allowed.append(self.action_dict["RIGHT"])
        actions_allowed = np.array(actions_allowed, dtype=int)
        return actions_allowed

    def _build_rewards(self):
        # agent rewards R[s,a]
        r_goal = 100 # reward for arriving at goal state (bottom-right corner)
        r_nongoal = -0.1 # penalty for not reaching goal state
        R = r_nongoal * np.ones(self.state_dim + self.action_dim, dtype=float)
        R[self.Ny - 2, self.Nx - 1, self.action_dict["DOWN"]] = r_goal # arrive from above
        R[self.Ny - 1, self.Nx - 2, self.action_dict["RIGHT"]] = r_goal # arrive from the left
        return R

In [53]:
class Agent:
    def __init__(self, env):
        # state and action dimension
        self.state_dim = env.state_dim
        self.action_dim = env.action_dim
        # Agent learning parameters
        self.epsilon = 1.0 # initial exploration probability
        self.epsilon_decay = 0.99 # epsilon decay after each episode
        self.alpha = 0.1 # learning rate
        self.gamma = 0.9 # reward discount factor
        # Initializing Q[s,a] table
        self.Q = np.zeros(self.state_dim + self.action_dim, dtype=float)

    def get_action(self, env):
        # Epsilon-greedy agent policy
        if random.uniform(0, 1) < self.epsilon:
            # explore
            return np.random.choice(env.allowed_actions())
        else:
            # exploit on allowed actions
            state = env.state
            actions_allowed = env.allowed_actions()
            Q_s = self.Q[state[0], state[1], actions_allowed]
            actions_greedy = actions_allowed[np.flatnonzero(Q_s == np.max(Q_s))]
            return np.random.choice(actions_greedy)

    def train(self, memory):

        # Update:
        # Q[s,a] <- (1 - alpha) * Q[s,a] + alpha * (R[s,a] + gamma * max(Q[s',:]))
        # R[s,a] = reward for taking action a from state s
        (state, action, state_next, reward, done) = memory
        sa = state + (action,)
        self.Q[sa] = (1 - self.alpha) * self.Q[sa] + self.alpha * (reward + self.gamma * np.max(self.Q[state_next]))

    def display_greedy_policy(self):
        # greedy policy
        greedy_policy = np.zeros((self.state_dim[0], self.state_dim[1]), dtype=int)
        for x in range(self.state_dim[0]):
            for y in range(self.state_dim[1]):
                greedy_policy[y, x] = np.argmax(self.Q[y, x, :])
        print("\nGreedy policy(y, x):")
        print(greedy_policy)
        print()

In [54]:
class Brain:
    def __init__(self, env, agent):
        self.env = env
        self.agent = agent

    def train_agent(self, N_episodes=500):
        # Train agent
        print("\nTraining agent...\n")
        for episode in range(N_episodes):
            # Generate an episode
            iter_episode, reward_episode = 0, 0
            state = self.env.reset() # starting state
            while True:
                action = self.agent.get_action(self.env)
                state_next, reward, done = self.env.step(action) # evolve state by action
                self.agent.train((state, action, state_next, reward, done)) # train agent
                iter_episode += 1
                reward_episode += reward
                if done:
                    break
                state = state_next # transition to next state
            
            # Decay agent exploration parameter
            self.agent.epsilon = max(self.agent.epsilon * self.agent.epsilon_decay, 0.01)

            if (episode == 0) or (episode + 1) % 10 == 0:
                print("[episode {}/{}] eps = {:.3F} -> iter = {}, rew = {:.1F}".format(
                    episode + 1, N_episodes, self.agent.epsilon, iter_episode, reward_episode))

            if episode == N_episodes - 1:
                self.agent.display_greedy_policy()
                for (key, val) in sorted(env.action_dict.items(), key=operator.itemgetter(1)):
                    print(" action['{}'] = {}".format(key, val))
                print()

In [55]:
# Settings
env = Environment(Ny=7, Nx=7)
agent = Agent(env)
brain = Brain(env, agent)

# Train agent
brain.train_agent()


Training agent...

[episode 1/500] eps = 0.990 -> iter = 126, rew = 87.5
[episode 10/500] eps = 0.904 -> iter = 240, rew = 76.1
[episode 20/500] eps = 0.818 -> iter = 64, rew = 93.7
[episode 30/500] eps = 0.740 -> iter = 94, rew = 90.7
[episode 40/500] eps = 0.669 -> iter = 56, rew = 94.5
[episode 50/500] eps = 0.605 -> iter = 22, rew = 97.9
[episode 60/500] eps = 0.547 -> iter = 32, rew = 96.9
[episode 70/500] eps = 0.495 -> iter = 30, rew = 97.1
[episode 80/500] eps = 0.448 -> iter = 26, rew = 97.5
[episode 90/500] eps = 0.405 -> iter = 12, rew = 98.9
[episode 100/500] eps = 0.366 -> iter = 28, rew = 97.3
[episode 110/500] eps = 0.331 -> iter = 28, rew = 97.3
[episode 120/500] eps = 0.299 -> iter = 12, rew = 98.9
[episode 130/500] eps = 0.271 -> iter = 14, rew = 98.7
[episode 140/500] eps = 0.245 -> iter = 16, rew = 98.5
[episode 150/500] eps = 0.221 -> iter = 16, rew = 98.5
[episode 160/500] eps = 0.200 -> iter = 14, rew = 98.7
[episode 170/500] eps = 0.181 -> iter = 16, rew = 98.5